In [1]:
from __future__ import division
import codecs
from collections import Counter
import pandas as pn
import random
import numpy as np
import pickle

filename='new_pairs'
class_num=35
iter_num=5
use_pickle=True
store_matrices=False

# open file - read verb nouns    
with codecs.open(filename,"r","utf-8") as f:
    f_lines = f.readlines()


# split verb and noun
f_lines = [f_lines[i].encode("utf-8").lower().rstrip().split(None,1) for i in range(len(f_lines))]
f_lines = [i for i in f_lines if '_s_' in i[0] or '_so_' in i[0]]
verbs=[]
for i in f_lines:
    verbs.append(i[0].lower().rstrip())
verbs_freqs=Counter(verbs)
f_lines = [i for i in f_lines if not (verbs_freqs[i[0]]<11)]
     
# first implementation of the frequencies counter        
pairs = [str(f_lines[i][0])+' '+str(f_lines[i][1]) for i in range(len(f_lines))]
pairs = Counter(pairs)

# verbs and nouns lists with all the verbs and nouns
verbs = []
nouns = []
for i in f_lines:
    verbs.append(i[0].lower().rstrip())
    nouns.append(i[1].lower().rstrip())

verbs_freqs = Counter(verbs)  
nouns_freqs = Counter(nouns)
unique_verbs = list(set(verbs))
unique_nouns = list(set(nouns))

verbs_dict = {}
nouns_dict = {}
for i in f_lines:
    i[0] = i[0].lower().rstrip()
    i[1] = i[1].lower().rstrip()
    
    if i[0] in verbs_dict:
        if i[1] in verbs_dict[i[0]]:
            verbs_dict[i[0]][i[1]] += 1
        else:
            verbs_dict[i[0]].update({i[1] : 1})
        
    else:
        verbs_dict[i[0]] = {i[1] : 1}
        
    if i[1] in nouns_dict:
        if i[0] in nouns_dict[i[1]]:
            nouns_dict[i[1]][i[0]] += 1
        else:
            nouns_dict[i[1]].update({i[0] : 1})
        
    else:
        nouns_dict[i[1]] = {i[0] : 1}


if use_pickle==True:
    
    file_p_c='second_phase_final_pairs/p_c_50.pkl'
    file_p_n_c='second_phase_final_pairs/p_n_c_50'
    file_p_v_c='second_phase_final_pairs/p_v_c_50'
    file_pairs_classes='second_phase_final_pairs/pairs_classes50'

    p_c = pickle.load(open(file_p_c, 'rb'))
    p_n_c = pickle.load(open(file_p_n_c, 'rb'))
    p_v_c = pickle.load(open(file_p_v_c, 'rb'))
    pairs_classes = pickle.load(open(file_pairs_classes, 'rb'))
    
else:
        
    # EM for verb-noun-pair classes

    pairs_classes = pn.DataFrame.from_items([(pair,[pairs[pair]/class_num]*class_num) for pair in pairs],orient='index', columns=[i for i in range(0,class_num)])

    p_v_c = pn.DataFrame(np.random.uniform(0,1,size=(class_num,len(set(verbs)))), columns=[v for v in set(verbs)])

    p_n_c = pn.DataFrame(np.random.uniform(0,1,size=(class_num,len(set(nouns)))), columns=[n for n in set(nouns)])


    p_c = [1.0/class_num]*class_num
    
    # E-Step

    for i in range(1,iter_num+1):
        print i

        for pair in pairs:

            prod_classes_verbs_nouns = [a*b*c for a,b,c in zip(p_c,p_v_c[pair.split(None,1)[0]].tolist(),p_n_c[pair.split(None,1)[1]].tolist())]
            norm = sum(prod_classes_verbs_nouns)

            for class_ in range(len(p_c)):

                pairs_classes.set_value(pair,class_,(p_c[class_] * p_v_c[pair.split(None,1)[0]][class_] * p_n_c[pair.split(None,1)[1]][class_])/norm)


        # M Step

        # Calculate sum_over_y(f(y)*p_theta(x|y))
        y=[]
        for class_ in xrange(class_num):
            probs=[]
            for verb in verbs_dict:
                for noun in verbs_dict[verb]:
                    probs.append(verbs_dict[verb][noun] * pairs_classes.at[str(verb) + ' ' + str(noun),class_])
            y.append(sum(probs))

        # M(theta_vc)
        for class_ in xrange(class_num):
            for verb in verbs_dict:
                probs=[]
                for noun in verbs_dict[verb]:
                    probs.append(verbs_dict[verb][noun] * pairs_classes.at[str(verb) + ' ' + str(noun),class_])
                p_v_c.at[class_,str(verb)] = sum(probs) / y[class_]

        # M(theta_nc)
        for class_ in xrange(class_num):
            for noun in nouns_dict:
                probs=[]
                for verb in nouns_dict[noun]:
                    probs.append(nouns_dict[noun][verb] * pairs_classes.at[str(verb) + ' ' + str(noun),class_])
                p_n_c.at[class_,str(noun)] = sum(probs) / y[class_]

        # M(theta_c)
        for class_ in xrange(class_num):
            p_c[class_] = y[class_] / len(pairs_classes)

        # Store results
        
        if store_matrices==True:
        
            p_v_c.to_pickle('p_v_c/p_v_c_'+str(i))
            p_n_c.to_pickle('p_n_c/p_n_c_'+str(i))
            with open('p_c/p_c_'+str(i)+'.pkl','wb') as f:
                pickle.dump(p_c,f)
            pairs_classes.to_pickle('pairs_classes/pairs_classes'+str(i))
        


In [2]:
# Get hard verb classes

v_c = {}

for verb in verbs_dict:
    class_=p_v_c[verb].idxmax()
    if class_ in v_c:
        v_c[class_].update({verb:p_v_c[verb].max()})
    else:
        v_c[class_]={verb:p_v_c[verb].max()}

for class_ in xrange(class_num):    
    v_c[class_]=sorted(v_c[class_].items(),key=lambda x: x[1], reverse=True)[:20]

# Show highest ranked verbs for each class (soft classes)
#for class_ in xrange(class_num):
#    print class_
#    print p_v_c.iloc[class_].sort_values(ascending=False)[:20]

# Get hard noun classes

n_c = {}

for noun in nouns_dict:
    class_=p_n_c[noun].idxmax()
    if class_ in n_c:
        n_c[class_].update({noun:p_n_c[noun].max()})
    else:
        n_c[class_]={noun:p_n_c[noun].max()}

for class_ in xrange(class_num):    
    n_c[class_]=sorted(n_c[class_].items(),key=lambda x: x[1], reverse=True)[:20]

In [3]:
v_c

{0: [('go_so_dobj', 0.073885993032784356),
  ('open_so_dobj', 0.072349925317457209),
  ('run_so_dobj', 0.062703502255628818),
  ('use_so_dobj', 0.058788710054602768),
  ('hold_so_dobj', 0.04246147540754245),
  ('turn_so_dobj', 0.040884762632863836),
  ('see_so_dobj', 0.030187420528930489),
  ('open_s_nsubj', 0.029904227203086695),
  ('lead_so_dobj', 0.029303676764539578),
  ('come_so_dobj', 0.029166161552264844),
  ('close_so_dobj', 0.027037058782250264),
  ('close_s_nsubj', 0.014110775594048887),
  ('walk_so_dobj', 0.013261630616256912),
  ('push_so_dobj', 0.011099763636652922),
  ('pull_so_dobj', 0.011021884134609692),
  ('return_so_dobj', 0.010910026362704621),
  ('watch_so_dobj', 0.010644085971026049),
  ('pass_so_dobj', 0.010610386312866772),
  ('swing_so_dobj', 0.0061247022605636766),
  ('fight_so_dobj', 0.0059220711456003487)],
 1: [('follow_s_nsubj', 0.01576367261115063),
  ('add_so_nsubj', 0.011327374820641077),
  ('suggest_so_nsubj', 0.010320951204381012),
  ('indicate_so_nsu

In [4]:
n_c

{0: [('way', 0.058846948401892835),
  ('door', 0.055489501234725791),
  ('eye', 0.048184743345991723),
  ('home', 0.041814111832527354),
  ('round', 0.041518252789400953),
  ('face', 0.01503393476326751),
  ('lip', 0.0081336037583027831),
  ('mouth', 0.0079110386358945562),
  ('back', 0.0064352789390360781),
  ('window', 0.0058732949664776161),
  ('mile', 0.005814602401085116),
  ('teeth', 0.0047326603280592825),
  ('shop', 0.0042733698518247799),
  ('gate', 0.0040726127946595757),
  ('corner', 0.0035582055112007172),
  ('other', 0.0033131707936324252),
  ('page', 0.0033040886524698192),
  ('length', 0.0032377977561181114),
  ('past', 0.0031869940670093367),
  ('parallel', 0.0029921857145403861)],
 1: [('that', 0.21183151159904184),
  ('fact', 0.0033373235854701039),
  ('discovery', 0.0010258054194198472),
  ('former', 0.00041166665141961978),
  ('revelation', 0.00037350844375103439),
  ('main', 0.00033427028583263388),
  ('inclusion', 0.00032858102680538679),
  ('gateway', 0.000268396

In [7]:
# Getting intransitive verbs

intr_verbs={}
for verb in verbs_dict:
    if '_s_' in verb:
        intr_verbs[verb] = verbs_dict[verb]

# Calc p(n)

p_n = {}
for verb in intr_verbs:
    for noun in intr_verbs[verb]:
        tmp=[]
        for class_ in xrange(class_num):
            tmp.append(p_c[class_]*p_n_c.at[class_,noun])
        if verb in p_n:
            p_n[verb].update({noun:sum(tmp)})
        else:
            p_n[verb]={noun:sum(tmp)}

# Estimate           

verb_noun_rank = {}
verb_noun_value={}
threshold=10
for verb in intr_verbs:
    tmp=[]
    tmp2=[]
    verb_class = p_v_c[verb].idxmax()
    for noun in intr_verbs[verb]:
        p_theta=(p_c[verb_class] * p_n_c.at[verb_class,noun]) / p_n[verb][noun]
        tmp.append(intr_verbs[verb][noun] * p_theta)
        tmp2.append(intr_verbs[verb][noun])
        if verb in verb_noun_rank:
            verb_noun_rank[verb].update({noun:intr_verbs[verb][noun] * p_theta})
        else:
            verb_noun_rank[verb]={noun:intr_verbs[verb][noun] * p_theta}
    verb_noun_rank[verb]=sorted(verb_noun_rank[verb].items(),key=lambda x: x[1], reverse=True)[:threshold]
    verb_noun_value[verb]=(sum(tmp)/sum(tmp2))

In [8]:
verb_noun_rank

{'wake_s_nsubj': [('<pro>', 12.055140745602635),
  ('waking', 1.0),
  ('finnegans', 1.0),
  ('frankie', 0.94149671099971532),
  ('sara', 0.77598062094482101),
  ('reveller', 0.47577656880408858),
  ('mungo', 0.42978541385876345),
  ('marian', 0.42592052996512592),
  ('philip', 0.34129017109849075),
  ('cameron', 0.24283310631946389)],
 'prepare_s_nsubj': [('<pro>', 2.6626496222508615),
  ('papillon', 1.0),
  ('carlists', 1.0),
  ('13.2', 1.0),
  ('chatam', 1.0),
  ('krantz', 1.0),
  ('woodstork', 1.0),
  ('frome', 0.50465638083753184),
  ('mcfall', 0.48180474669357221),
  ('hut', 0.36013512539331255)],
 'bore_s_nsubj': [('da', 0.8487675797451355),
  ('<pro>', 0.34220382017240514),
  ('dickie', 0.21960753056159149),
  ('great', 0.16778256123671606),
  ('visitor', 0.11369179757307615),
  ('tremayne', 0.10936516701260754),
  ('hundred', 0.10375927561451742),
  ('galley', 0.097717859660710768),
  ('meeting', 0.047260435295882454),
  ('parent', 0.047066711303259755)],
 'conflict_s_nsubj': [

In [9]:
# Getting transitive verbs

trans_verbs={}
for verb_s in verbs_dict:
    if '_so_' in verb_s and 'nsubj' in verb_s:
        for verb_o in verbs_dict:
            if '_so_' in verb_o and 'dobj' in verb_o:
                if verb_s.split('_')[0] == verb_o.split('_')[0]:
                    trans_verbs[(str(verb_s),str(verb_o))]=[(verbs_dict[verb_s],verbs_dict[verb_o])]

# Get p(n_1,n_2)

p_n1_n2 = {}

for verb in trans_verbs:
    tmp=[]
    for noun1 in trans_verbs[verb][0][0]:
        for noun2 in trans_verbs[verb][0][1]:
            for class1 in xrange(class_num):
                for class2 in xrange(class_num):
                    tmp.append((p_c[class1]*p_c[class2])*p_n_c.at[class1,noun1]*p_n_c.at[class2,noun2])
            if verb in p_n1_n2:
                p_n1_n2[verb].update({(noun1,noun2):sum(tmp)})
            else:
                p_n1_n2[verb]={(noun1,noun2):sum(tmp)}

# Estimate

verb_tr_noun_rank={}
threshold=10
for verb in p_n1_n2:
    verb_class1=p_v_c[verb[0]].idxmax()
    verb_class2=p_v_c[verb[1]].idxmax()
    for noun1,noun2 in p_n1_n2[verb].keys():
        p_theta=(p_c[verb_class1] * p_c[verb_class2] * p_n_c.at[verb_class1,noun1] * p_n_c.at[verb_class2,noun2]) / p_n1_n2[verb][(noun1,noun2)] # Not correct
        if verb in verb_tr_noun_rank:
            verb_tr_noun_rank[verb].update({(noun1,noun2):verbs_dict[verb[0]][noun1]+verbs_dict[verb[1]][noun2] * p_theta})
        else:
            verb_tr_noun_rank[verb]={(noun1,noun2):verbs_dict[verb[0]][noun1]+verbs_dict[verb[1]][noun2] * p_theta}
    verb_tr_noun_rank[verb]=sorted(verb_tr_noun_rank[verb].items(),key=lambda x: x[1], reverse=True)[:]